# Plotting routine for the 2020 April 19 Event - SolO

In [ ]:
test = 'test'

In [44]:
%load_ext autoreload

%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pickle as p

import datetime as datetime
from datetime import timedelta
import seaborn as sns
import pandas as pd
import os
#import copy as cp

import multiprocessing
import py3dcore_h4c as py3dcore_h4c

import py3dcore_h4c.fluxplot as fp

plt.switch_backend('agg')


import warnings
warnings.filterwarnings('ignore')

import logging

logging.basicConfig(level=logging.INFO)
logging.getLogger("heliosat.spice").setLevel("WARNING")
logging.getLogger("heliosat.spacecraft").setLevel("WARNING")
#logging.getLogger("sunpy").setLevel("WARNING")
#logging.getLogger("py3dcore_h4c.fluxplot").setLevel("WARNING")

#print(sns.__version__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### setting the times for py3DCORE_h4c

In [46]:
t_launch = datetime.datetime(2020, 4, 15, 23, tzinfo=datetime.timezone.utc) # 

t_s = datetime.datetime(2020, 4, 19, 15, tzinfo=datetime.timezone.utc) 
t_e = datetime.datetime(2020, 4, 20, 3, tzinfo=datetime.timezone.utc)

t_fit = [
        datetime.datetime(2020, 4, 19, 17, tzinfo=datetime.timezone.utc),
        datetime.datetime(2020, 4, 19, 18, tzinfo=datetime.timezone.utc),
        datetime.datetime(2020, 4, 19, 19, tzinfo=datetime.timezone.utc),
        datetime.datetime(2020, 4, 19, 20, tzinfo=datetime.timezone.utc),
        datetime.datetime(2020, 4, 19, 21, tzinfo=datetime.timezone.utc),
        datetime.datetime(2020, 4, 19, 22, tzinfo=datetime.timezone.utc),
        datetime.datetime(2020, 4, 19, 23, tzinfo=datetime.timezone.utc),
        datetime.datetime(2020, 4, 20, 0, tzinfo=datetime.timezone.utc)
]


#### loading the pickle files

In [47]:
fittingPoints = []

fittingPoints = [t_fit[i].strftime("%m/%d/%Y, %H:%M:%S") for i in range(len(t_fit))]
print(fittingPoints)

['04/19/2020, 17:00:00', '04/19/2020, 18:00:00', '04/19/2020, 19:00:00', '04/19/2020, 20:00:00', '04/19/2020, 21:00:00', '04/19/2020, 22:00:00', '04/19/2020, 23:00:00', '04/20/2020, 00:00:00']


In [58]:
path = 'output/solo19042020_heeq_512_2/'

filepath = fp.loadpickle(path, number=-1)

In [59]:
filepath[:-7]

'output/solo19042020_heeq_512_2/09'

#### extracting parameters

In [60]:
res, allres, ind, meanparams = fp.get_params(filepath, give_mineps=True)

INFO:py3dcore_h4c.fluxplot:Retrieved the following parameters for the run with minimum epsilon:
INFO:py3dcore_h4c.fluxplot: --lon:  10.89
INFO:py3dcore_h4c.fluxplot: --lat:  -11.18
INFO:py3dcore_h4c.fluxplot: --inc:  347.91
INFO:py3dcore_h4c.fluxplot: --diameter 1 AU:  0.10
INFO:py3dcore_h4c.fluxplot: --aspect ratio:  2.82
INFO:py3dcore_h4c.fluxplot: --launch radius:  35.00
INFO:py3dcore_h4c.fluxplot: --launch speed:  445.68
INFO:py3dcore_h4c.fluxplot: --t factor:  -130.39
INFO:py3dcore_h4c.fluxplot: --expansion rate:  1.14
INFO:py3dcore_h4c.fluxplot: --magnetic field decay rate:  1.64
INFO:py3dcore_h4c.fluxplot: --magnetic field 1 AU:  13.16
INFO:py3dcore_h4c.fluxplot: --drag coefficient:  1.91
INFO:py3dcore_h4c.fluxplot: --background sw speed:  278.44


In [61]:
out = fp.get_ensemble_stats(filepath)

In [62]:
out.describe()

,lon,lat,inc,D1AU,delta,launch radius,launch speed,t factor,B1AU,gamma,vsw
count,512.000000,512.000000,512.000000,512.000000,512.000000,512.0,512.000000,512.000000,512.000000,512.000000,512.000000
mean,8.142889,-5.800335,348.634491,0.093718,4.315435,35.0,475.488159,-173.313339,13.378570,3.172815,281.902283
std,3.239545,5.730076,2.720269,0.014007,1.947945,0.0,96.269684,55.175217,0.569567,0.559718,4.862748
min,-5.065135,-22.954977,340.781433,0.065031,1.072401,35.0,350.094788,-249.957138,11.771422,1.286846,275.035553
25%,6.367187,-9.757552,346.888863,0.084636,2.786687,35.0,395.953758,-216.540874,13.007606,2.839502,278.219765
50%,8.408908,-5.888193,348.749786,0.092501,4.069546,35.0,447.869537,-183.316711,13.373053,3.271136,280.995544
75%,10.321652,-1.849578,350.563576,0.100562,5.619204,35.0,537.346939,-141.486816,13.767063,3.628299,284.233734
max,18.766407,8.934712,355.775818,0.157067,8.949588,35.0,747.773987,19.036564,15.051676,3.997135,300.031067


In [63]:
# write output of describe() to csv-file
out.describe().to_csv(filepath+'parameters.csv')

# write parameters of whole ensemble to txt-file
with open(filepath+'parameters_ensemble.txt', 'a') as f:
    dfAsString = out.to_string(header=True, index=True)
    f.write(dfAsString)

In [64]:
names = ['lon: ', 'lat: ', 'inc: ', 'diameter 1 AU: ', 'aspect ratio: ', 'launch radius: ', 'launch speed: ', 
         't factor: ', 'expansion rate: ', 'magnetic field decay rate: ', 'magnetic field 1 AU: ', 'drag coefficient: ', 
         'background sw speed: ']
 
# write output of describe() to txt-file    
with open(filepath+'parameters_summary.txt', 'a') as f:
    dfAsString = out.describe().to_string(header=True, index=True)
    f.write(dfAsString)   
    f.write('\n')
    f.write('Fitting points:')
    f.write(str(fittingPoints))
    f.write('\n')
    f.write('Parameters for run with min(eps):')
    f.write('\n')
    for count, name in enumerate(names):
        f.write(" --{} {:.2f}".format(name, res[count+1]))
        f.write('\n')

#### loading and plotting the fitting results

In [65]:
t_s = datetime.datetime(2020, 4, 19, 3, tzinfo=datetime.timezone.utc) 
t_e = datetime.datetime(2020, 4, 20, 23, tzinfo=datetime.timezone.utc)

In [66]:
fp.fullinsitu(observer='solo', t_fit=t_fit, start=t_s, end=t_e, filepath=filepath, custom_data='solo_2020apr.p', best=True, ensemble=True, 
              mean=False, save_fig=True, legend=True, fixed=None, max_index=128, title=True, fit_points=True)

INFO:py3dcore_h4c.fluxplot:No iparams_arr given, using parameters for run with minimum eps.
INFO:py3dcore_h4c.fitter.base:Using custom datafile: solo_2020apr.p


In [67]:
fp.scatterparams(filepath)

In [ ]:
print(out['lon'].mean())
print(out['lon'].std())
print(out['launch radius'].std())
print(out['launch speed'].std())

#### Plotting the fluxrope and spacecraft positions

In [ ]:
#measurement times 
#tm0 =  t_launch + datetime.timedelta(days=1.5)
tm0 =  datetime.datetime(2022, 9, 5, 16)
tm1 =  t_launch + datetime.timedelta(days=1.7)
tm2 =  t_launch + datetime.timedelta(days=3.5)

fig = fp.full3d(spacecraftlist=['solo', 'psp'], planetlist=['Earth'], t=tm1, filepath=filepath)

### make fluxrope 3d movie

In [ ]:
def make_frame(k):
    tm0 = datetime.datetime(2022, 9, 5, 19)
    t = tm0 + k*datetime.timedelta(hours=1)
    
    fig = fp.full3d(spacecraftlist=['solo', 'psp'], planetlist =['Earth'],
              t = t, filepath = filepath, save_fig = False)
    frmstr = '%05i' % (k) 
    plt.savefig(filepath[:-7] + 'movie_3d/'+frmstr+'.jpg',dpi=200)  
    return fig
    

In [ ]:
inn=[i for i in range(100)]


mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame, inn[0:20])
mpool.close()
mpool.join()

print('done 1/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame, inn[20:40])
mpool.close()
mpool.join()

print('done 2/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame, inn[40:60])
mpool.close()
mpool.join()

print('done 3/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame, inn[40:80])
mpool.close()
mpool.join()

print('done 4/4')

In [ ]:
os.system('ffmpeg -r 25 -i '+filepath[:-7]+'movie_3d/%05d.jpg -b 5000k -r 25 '+filepath[:-7]+'movie_3d/full_3d_movie.mp4 -y -loglevel quiet')

### plot fluxrope 3d multiview

In [ ]:
#measurement times 
#tm0 =  t_launch + datetime.timedelta(days=1.5)
tm0 =  datetime.datetime(2022, 9, 7, 1)
tm1 =  t_launch + datetime.timedelta(days=1.7)
tm2 =  t_launch + datetime.timedelta(days=3.5)

fig = fp.full3d_multiview(t_launch = t_launch, filepath=filepath)

### make fluxrope 3d multiview movie

In [ ]:
def make_frame2(k):
    tm0 = datetime.datetime(2022, 9, 5, 19)
    t = tm0 + k*datetime.timedelta(hours=1)
    frametime = k
    
    fig = fp.full3d_multiview_movie(t_launch = tm0, t = t, filepath=filepath,
                                    frametime=k)
    frmstr = '%05i' % (k) 
    plt.savefig(filepath[:-7] + 'movie_3d_multiview/'+frmstr+'.jpg',dpi=200)  
    return fig

In [ ]:
inn=[i for i in range(100)]


mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[0:20])
mpool.close()
mpool.join()

print('done 1/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[20:40])
mpool.close()
mpool.join()

print('done 2/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[40:60])
mpool.close()
mpool.join()

print('done 3/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[40:80])
mpool.close()
mpool.join()

print('done 4/4')

In [ ]:
os.system('ffmpeg -r 25 -i '+filepath[:-7]+'movie_3d_multiview/%05d.jpg -b 5000k -r 25 '+filepath[:-7]+'movie_3d/full_3d_multiview_movie.mp4 -y -loglevel quiet')

# Used for Debug

In [ ]:
TP_A =  t_launch + datetime.timedelta(hours=2)
TP_B =  t_launch + datetime.timedelta(hours=40)


C_A = "xkcd:red"
C_B = "xkcd:blue"
    
sns.set_style('whitegrid')

fig = plt.figure(figsize=(15, 11),dpi=100)

ax1 = plt.subplot2grid((2, 3), (0, 0),rowspan=2,colspan=2,projection='3d')  
ax2 = plt.subplot2grid((2, 3), (0, 2),projection='3d')  
ax3 = plt.subplot2grid((2, 3), (1, 2),projection='3d')  

model_obj = fp.returnmodel(filepath)


fp.plot_configure(ax1, view_azim=150, view_elev=25, view_radius=.2,light_source=True) #view_radius=.08

fp.plot_3dcore(ax1, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax1, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax1,0.31,-0.25,0.0,-0.2)


fp.plot_configure(ax2, view_azim=165-90, view_elev=90, view_radius=.08,light_source=True)
    
fp.plot_3dcore(ax2, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax2, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax2,0.26,-0.41,0.08,0.0)



fp.plot_configure(ax3, view_azim=65, view_elev=-5, view_radius=.01,light_source=True)
    
fp.plot_3dcore(ax3, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax3, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax3,0.26,-0.41,0.08,0.0)

